In [1]:
import os
os.chdir( "../")

In [2]:
#BEST PARAMS

DICT_RP3 = {'topK': 29, 'alpha': 0.33723025040409343, 'beta': 0.15542352567862933, 'normalize_similarity': True}

DICT_SLIMEN = {'topK': 638, 'l1_ratio': 0.026639245044358113, 'alpha': 0.001535792379351249} # 0.05018798797659525

DICT_IALS = {'use_gpu': True, 'num_threads' : 1, 'num_factors': 128, 'epochs': 478, 'confidence_scaling': 'linear', 'alpha': 3.358295366834473, 'epsilon': 7.295986722921583, 'reg': 6.292360640597378e-05}
# 0.03486293010333361

DICT_KNN = {'topK': 19, 'shrink': 28, 'similarity': 'tversky', 'normalize': True, 'tversky_alpha': 0.0, 'tversky_beta': 1.3672226785339947}

DICT_HYBIRD = {'alpha1': 0.6464483146524337, 'alpha2': 0.7533801103113421, 'alpha3': 0.032576355148477586, 'min_interaction': 1}

#TRIAL MATRIXES + IALS

In [3]:
import pandas as pd
import numpy as np

path = "Dataset/data_train.csv"
df = pd.read_csv(filepath_or_buffer=path,
                               sep=",",
                               header=1,
                               engine='python',
                               names=['UserID', 'ItemID', 'Interaction'])


df

,UserID,ItemID,Interaction
0,1,15,1.0
1,1,16,1.0
2,1,133,1.0
3,1,161,1.0
4,1,187,1.0
...,...,...,...
478724,13024,13605,1.0
478725,13024,13823,1.0
478726,13024,15122,1.0
478727,13024,18185,1.0


In [4]:
target_users = pd.read_csv("Dataset/data_target_users_test.csv")
target_users.columns = ["UserID"]

In [5]:
user_ids = df["UserID"].unique().tolist()
user2user_encoded = {x: i for i, x in enumerate(user_ids)}
userencoded2user = {i: x for i, x in enumerate(user_ids)}
item_ids = df["ItemID"].unique().tolist()
item2item_encoded = {x: i for i, x in enumerate(item_ids)}
item_encoded2item = {i: x for i, x in enumerate(item_ids)}
df["User"] = df["UserID"].map(user2user_encoded)
df["Item"] = df["ItemID"].map(item2item_encoded)

num_users = len(user2user_encoded)
num_items = len(item_encoded2item)
df["Interaction"] = df["Interaction"].values.astype(np.float32)

# min and max ratings will be used to normalize the ratings later
min_rating = 0.0
max_rating = max(df["Interaction"])

print(
    "Number of users: {}, Number of Items: {}, Min rating: {}, Max rating: {}".format(
        num_users, num_items, min_rating, max_rating
    )
)

Number of users: 12638, Number of Items: 22222, Min rating: 0.0, Max rating: 1.0


In [6]:
from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_val_score, KFold
import numpy as np
import scipy.sparse as sps
from Data_manager.split_functions.split_train_validation_random_holdout import split_train_in_two_percentage_global_sample

urm_all = sps.coo_matrix((df["Interaction"].values, 
                          (df["User"].values, df["Item"].values)))

In [7]:
userId_unique = df["UserID"].unique()
itemId_unique = df["ItemID"].unique()

In [8]:
num_users = len(userId_unique)
num_items = len(itemId_unique)

In [9]:
from Recommenders.Recommender_import_list import *
from Recommenders.KNN.ItemKNNSimilarityHybridRecommender import ItemKNNSimilarityHybridRecommender
from Recommenders.Hybrid.GeneralizedLinearHybridRecommenderCold import GeneralizedLinearHybridRecommenderCold
from Recommenders.MatrixFactorization.ImplicitIALSRecommender import ImplicitALSRecommender

2024-01-09 19:16:09.083716: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2024-01-09 19:16:09.083751: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.


In [10]:
p3a_single = P3alphaRecommender(urm_all)
p3a_single.fit(**{'alpha':0.740243876372529,'topK':41})

rp3beta = RP3betaRecommender(urm_all)
rp3beta.fit(**{'topK': 29, 'alpha': 0.33723025040409343, 'beta': 0.15542352567862933, 'normalize_similarity': True})

knn_single = ItemKNNCFRecommender(urm_all)
knn_single.fit(**{'topK': 19, 'shrink': 28, 'similarity': 'tversky', 'normalize': True, 'tversky_alpha': 0.0, 'tversky_beta': 1.3672226785339947})

slim_single = MultiThreadSLIM_SLIMElasticNetRecommender(urm_all)
slim_single.fit(workers=12,**{'alpha':0.0008361537677496276,'topK':3334, 'l1_ratio':0.09976758088492818})

ials_single = ImplicitALSRecommender(urm_all)
ials_single.fit(use_gpu=True, num_threads=1,**{'num_factors': 401, 'reg': 0.0010579931024405477,'epsilon':0.10666578585393433, 'epochs':1204, 'alpha': 7.139858098793055, 'confidence_scaling': 'linear'})

P3alphaRecommender: Similarity column 22222 (100.0%), 6515.10 column/sec. Elapsed time 3.41 sec
RP3betaRecommender: Similarity column 22222 (100.0%), 6395.15 column/sec. Elapsed time 3.47 sec
Similarity column 22222 (100.0%), 9528.62 column/sec. Elapsed time 2.33 sec


100%|█████████▉| 22216/22222 [00:40<00:00, 551.90it/s] 


Using gpu: True


  0%|          | 0/1204 [00:00<?, ?it/s]

In [11]:
hybrid = GeneralizedLinearHybridRecommender(urm_all, recommenders=[ials_single, p3a_single, rp3beta, knn_single, slim_single])
hybrid.fit(alphas=[0.17604900089728054, 0.09467027692420081, 0.8285908265891366, 0.10218434512122093, 0.976403114543983])

In [12]:
item_popularity_encoded = np.ediff1d(urm_all.tocsc().indptr)
item_popularity_encoded = np.sort(item_popularity_encoded)

tar_users = target_users["UserID"].astype(int)
topPop_encoded = item_popularity_encoded[-10:]

submission = []

print(np.unique(df["UserID"].values))

for index, user in enumerate(tar_users):
    if (user not in df["UserID"].values):
        item_list_encoded = topPop_encoded
    else:
        item_list_encoded = hybrid.recommend(user2user_encoded[user])[:10]
    item_list = []
    for item_encoded in item_list_encoded:
        item_list.append(item_encoded2item[item_encoded])
    submission.append((user, item_list))


def write_submission(submissions):
    with open("./submission_hybrid_big.csv", "w") as f:
        f.write("user_id,item_list\n")
        for user_id, items in submissions:
            f.write(f"{user_id},{' '.join([str(item) for item in items])}\n")
            
write_submission(submission)

[    1     2     3 ... 13022 13023 13024]


In [21]:
def compareDataframes(name1, name2):

    path = "/home/japo/RecSys-Challenge/"

    #Load the csv
    csv1 = pd.read_csv(path + name1)
    csv2 = pd.read_csv(path + name2)

    df1 = pd.DataFrame(csv1)
    df1['item_list'] = df1['item_list'].apply(lambda x: [int(item) for item in x.split()])
    df2 = pd.DataFrame(csv2)
    df2['item_list'] = df2['item_list'].apply(lambda x: [int(item) for item in x.split()])

    if df1.shape == df2.shape:
        common_items = []
        for index, row1 in df1.iterrows():
            row2 = df2.iloc[index]
            items1 = set(row1['item_list'])
            items2 = set(row2['item_list'])
            common_items.append(len(items1.intersection(items2)))
            print(items1.intersection(items2))
        count = 0
        for row in common_items:
            count += row
        res = count/(len(common_items)*10)
        return res
    else:
        print("The two dataframes have different shape")

compareDataframes("submission_best.csv", "submission_tris_hybrid_our_params.csv")

{515, 36, 101, 403, 694, 506, 123}
{196, 102, 1095, 12, 47, 656, 1522, 50, 28}
{1281, 259, 536, 857, 59, 4252}
{7, 139, 171, 50, 249, 314, 28, 254}
{1570, 131, 1220, 1511, 77, 5138, 116}
{35, 104, 9, 874, 395, 168, 886, 184, 88}
{480, 451, 3916, 210, 722, 600, 443}
{4802, 2821, 17799, 10108, 2282, 4535, 9018, 316}
{3905, 1668, 2565, 1446, 1767, 561, 2423, 1816, 2617}
{514, 67, 40, 58, 955, 31}
{4704, 2242, 325, 261, 329, 2582, 408, 2781}
{2403, 1316, 11747, 11174, 11016, 11048, 9813, 9817, 10234}
{1050, 164, 297, 2671, 90, 571, 1244, 2909}
{13250, 388, 236, 6957, 653, 1107, 308, 220}
{1569, 1346, 7, 617, 946, 1463, 9752, 1692, 29, 3198}
{98, 43, 11, 44, 47, 80, 50, 19, 57}
{192, 2, 332, 973, 3226, 766, 255}
{9442, 9482, 8044, 3571, 252, 8473, 154, 764}
{134, 10, 242, 467, 279, 376, 313, 573}
{2, 873, 786, 405, 1111, 152, 14140, 189}
{1290, 3351, 754, 82, 2295, 6111, 8444, 1279}
{10595, 13156, 2247, 4937, 3852, 22124, 18934}
{51, 1309}
{41, 139, 269, 1005, 314, 254}
{325, 133, 39, 136, 

0.7796085278441462